In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/"))

import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
from multiprocessing import Process, Pipe
import time
# Any results you write to the current directory are saved as output.
%matplotlib inline

['mulliken_charges.csv', 'test.csv', 'structures.csv', 'dipole_moments.csv', 'potential_energy.csv', 'train.csv', 'scalar_coupling_contributions.csv', 'magnetic_shielding_tensors.csv', 'sample_submission.csv']


In [4]:
train_df = pd.read_csv('../input/train.csv')
structure_df = pd.read_csv('../input/structures.csv')
molecule_name = structure_df['molecule_name'].unique()
type_name = train_df['type'].unique()

In [3]:
jobs = 10

calculate euclidean destance

In [11]:
splitnum = len(train_df)//jobs+1
def calc_distance(index, send):
    data = train_df.values[index*splitnum:(index+1)*splitnum]
    euclid_dist = np.zeros((len(data),1))
    molecule = None
    if index==0:
        for i,row in tqdm(enumerate(data),total=len(data)):
            if row[1] != molecule:
                molecule = row[1]
                molecule_data = structure_df[structure_df['molecule_name']==molecule]
            euclid_dist[i,0] = np.linalg.norm(molecule_data.values[row[2],3:6]-molecule_data.values[row[3],3:6])
    else:
        for i,row in enumerate(data):
            if row[1] != molecule:
                molecule = row[1]
                molecule_data = structure_df[structure_df['molecule_name']==molecule]
            euclid_dist[i,0] = np.linalg.norm(molecule_data.values[row[2],3:6]-molecule_data.values[row[3],3:6])
    send.send(euclid_dist)
    send.close()

In [12]:
p_jobs = []
pipes = []
for i in range(jobs):
    get_rev, send_rev = Pipe()
    p = Process(target=calc_distance, args=(i,send_rev))
    pipes.append(get_rev)
    p_jobs.append(p)
    p.start()
    
result = [x.recv() for x in pipes]

for p in p_jobs:
    p.join()

result = np.concatenate(result)

100%|██████████| 465815/465815 [19:29<00:00, 398.25it/s]  


In [13]:
pd.to_pickle(result, 'distance.pkl')

calculate H number

In [9]:
splitnum = len(molecule_name)//jobs+1
def calc_h(index, send):
    data = molecule_name[index*splitnum:(index+1)*splitnum]
    h_count = np.zeros((len(data),1))
    if index == 0:
        for i,m in tqdm(enumerate(data),total=len(data)):
            h_count[i,0] = np.where(structure_df[structure_df['molecule_name']==m]['atom'].values=='H')[0].shape[0]
    else:
        for i,m in enumerate(data):
            h_count[i,0] = np.where(structure_df[structure_df['molecule_name']==m]['atom'].values=='H')[0].shape[0]
    
    send.send(calc_h)
    send.close()

In [10]:
p_jobs = []
pipes = []
for i in range(jobs):
    get_rev, send_rev = Pipe()
    p = Process(target=calc_h, args=(i,send_rev))
    pipes.append(get_rev)
    p_jobs.append(p)
    p.start()
    
h_num = [x.recv() for x in pipes]

for p in p_jobs:
    p.join()

h_num = np.concatenate(num)

  0%|          | 30/13078 [00:03<25:06,  8.66it/s]

KeyboardInterrupt: 

Process Process-30:
Traceback (most recent call last):
  File "/home/ohara_lab/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/ohara_lab/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-9-3c8fc0d18f3c>", line 10, in calc_h
    h_count[i,0] = np.where(structure_df[structure_df['molecule_name']==m]['atom'].values=='H')[0].shape[0]
  File "/home/ohara_lab/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/pandas/core/ops.py", line 1283, in wrapper
    res = na_op(values, other)
  File "/home/ohara_lab/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/pandas/core/ops.py", line 1143, in na_op
    result = _comp_method_OBJECT_ARRAY(op, x, y)
  File "/home/ohara_lab/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/pandas/core/ops.py", line 1122, in _comp_method_OBJECT_ARRAY
 

  File "/home/ohara_lab/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/pandas/core/ops.py", line 1122, in _comp_method_OBJECT_ARRAY
    result = libops.scalar_compare(x, y, op)
KeyboardInterrupt
Process Process-29:
Traceback (most recent call last):
  File "/home/ohara_lab/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/ohara_lab/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Process Process-28:
  File "<ipython-input-9-3c8fc0d18f3c>", line 10, in calc_h
    h_count[i,0] = np.where(structure_df[structure_df['molecule_name']==m]['atom'].values=='H')[0].shape[0]
  File "/home/ohara_lab/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/pandas/core/ops.py", line 1283, in wrapper
    res = na_op(values, other)
  File "/home/ohara_lab/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/pandas/c

In [ ]:
c = 0
stock = train_df.values[0,1]
result= np.zeros((len(train_df),1))
for i,row in tqdm(train_df.iterrows(),total=len(train_df)):
    if stock!=row[1]:
        c += 1
        result[i,0] = h_num[c,0]
        stock=row['molecule_name']
    else:
        result[i,0] = h_num[c,0]

In [ ]:
pd.to_pickle(result,'h_num.pkl')